In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


In [ ]:
%cd Soongsil-BERT/finetune

/content/drive/.shortcut-targets-by-id/19AmDlGex-hSZJyofrcVWB6uu3yQV-PSB/dataset/dataset/Soongsil-BERT/finetune


In [ ]:
args = AttrDict({
  "task": "hate-speech",
  "data_dir": "data",
  "ckpt_dir": "ckpt",
  "train_file": "train.tsv",
  "dev_file": "",
  "test_file": "val.tsv",
  "max_seq_len": 128,
  "num_train_epochs": 5,
  "weight_decay": 0.0,
  "gradient_accumulation_steps": 1,
  "adam_epsilon": 1e-8,
  "warmup_proportion": 0,
  "max_steps": -1,
  "max_grad_norm": 1.0,
  "no_cuda": False,
  "model_name_or_path": f"{model_name}",
  "output_dir": f"{model_name}-hate-speech-ckpt",
  "seed": 42,
  "train_batch_size": 32,
  "eval_batch_size": 64,
  "logging_steps": 300,
  "save_steps": 300,
  "learning_rate": 5e-5,
  "evaluate_test_during_training": True,
  "eval_all_checkpoints": True,
  "save_optimizer": False,
  "do_lower_case": False,
  "do_train": True,
  "do_eval": True,
  "model_type": "roberta"})

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

# Base Model (370M)

tokenizer = AutoTokenizer.from_pretrained("jason9693/soongsil-bert-base")

In [ ]:
%cd '/content/drive/MyDrive/dataset/dataset/Soongsil-BERT/finetune'
train_dataset = load_data(tokenizer, "train")
test_dataset = load_data(tokenizer, "test")

In [ ]:
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
  {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
   'weight_decay': args.weight_decay},
  {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 
   'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=int(t_total * args.warmup_proportion), num_training_steps=t_total)

In [ ]:
global_step = 69900
output_dir = "/content/drive/MyDrive/dataset/dataset/Soongsil-BERT/finetune/monologg/kobigbird-bert-base-hate-speech-ckpt"
ckpt_path = os.path.join(output_dir, f"checkpoint-{global_step}" )
    
model = AutoModelForSequenceClassification.from_pretrained(ckpt_path)
model.to(device)

BigBirdForSequenceClassification(
  (bert): BigBirdModel(
    (embeddings): BigBirdEmbeddings(
      (word_embeddings): Embedding(32500, 768, padding_idx=0)
      (position_embeddings): Embedding(4096, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BigBirdEncoder(
      (layer): ModuleList(
        (0-11): 12 x BigBirdLayer(
          (attention): BigBirdAttention(
            (self): BigBirdBlockSparseAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): BigBirdSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru

In [ ]:
tokenizer.save_pretrained(output_dir)



('/content/drive/MyDrive/dataset/dataset/Soongsil-BERT/finetune/monologg/kobigbird-bert-base-hate-speech-ckpt/tokenizer_config.json',
 '/content/drive/MyDrive/dataset/dataset/Soongsil-BERT/finetune/monologg/kobigbird-bert-base-hate-speech-ckpt/special_tokens_map.json',
 '/content/drive/MyDrive/dataset/dataset/Soongsil-BERT/finetune/monologg/kobigbird-bert-base-hate-speech-ckpt/vocab.json',
 '/content/drive/MyDrive/dataset/dataset/Soongsil-BERT/finetune/monologg/kobigbird-bert-base-hate-speech-ckpt/merges.txt',
 '/content/drive/MyDrive/dataset/dataset/Soongsil-BERT/finetune/monologg/kobigbird-bert-base-hate-speech-ckpt/added_tokens.json',
 '/content/drive/MyDrive/dataset/dataset/Soongsil-BERT/finetune/monologg/kobigbird-bert-base-hate-speech-ckpt/tokenizer.json')

In [ ]:
model.config.save_pretrained(output_dir)



In [ ]:
torch.save(model.state_dict(), os.path.join(output_dir, "pytorch_model.bin"))


In [ ]:
# model.save(output_dir)


In [ ]:
# checkpoints = list(
#       os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + "/**/" + "pytorch_model.bin", recursive=True))
# )

In [ ]:
# results = {}
# for checkpoint in checkpoints:
#   _gloabl_step = checkpoint.split("-")[-1]
#   eval_model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
#   eval_model.to(args.device)
#   eval_model.eval()
#   result = evaluate(eval_model, test_dataset, mode="test", _global_step=_gloabl_step)
#   result = dict((k +f"_{_gloabl_step}", v) for k, v in result.items())
#   results.update(result)

In [ ]:
# for key in sorted(results.keys()):
#   print(f"{key} = {results[key]}")

In [ ]:
!pip install transformers
!huggingface-cli login

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid.
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Ru

In [ ]:
!huggingface-cli upload /content/drive/MyDrive/dataset/dataset/Soongsil-BERT/finetune/monologg/kobigbird-bert-base-hate-speech-ckpt/checkpoint-69900 kcbert_hatespeech


usage: huggingface-cli <command> [<args>] repo
       [-h]
       {create}
       ...
huggingface-cli <command> [<args>] repo: error: argument {create}: invalid choice: 'upload' (choose from 'create')


In [ ]:
from huggingface_hub import HfApi
api = HfApi()

# Upload all the content from the local folder to your remote Space.
# By default, files are uploaded at the root of the repo
api.upload_folder(
    folder_path="/content/drive/MyDrive/dataset/dataset/Soongsil-BERT/finetune/monologg/kobigbird-bert-base-hate-speech-ckpt/checkpoint-69900",
    repo_id="Haaaaeun/kcbert_hatespeech",
    repo_type="model",
)

training_args.bin:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/457M [00:00<?, ?B/s]

'https://huggingface.co/Haaaaeun/kcbert_hatespeech/tree/main/'